**CRIANDO A DATABASE DW28**

In [ ]:
CREATE DATABASE DW28

**CRIANDO UMA TABELA DE LOG**

In [63]:
USE [DW28]
GO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='LOG_CARGAS' AND xtype='U')

CREATE TABLE LOG_CARGAS
 (ID INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
 NUMERO_ERRO INT,
 SERVERIDADE_ERRO INT,
 ESTADO_ERRO  INT,
 PROC_ERRO NVARCHAR(255),
 LINHA_ERRO INT,
 MSG_ERRO NVARCHAR(255),
 SITUACAO NVARCHAR(10),
 PROCESSO NVARCHAR(30),
 DATA_INI DATETIME,
 DATA_FIM DATETIME  
 )

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.025

**CRIANDO TABELA STG\_VENDAS**

In [73]:
USE [DW28]
GO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='STG_VENDAS' AND xtype='U')
CREATE TABLE [dbo].[STG_VENDAS](
	[ID_VENDA] [INT] NULL,
	[Segmento] [varchar](50) NULL,
	[Pais] [varchar](50) NULL,
	[Produto] [varchar](50) NULL,
	[TipoDesconto] [varchar](50) NULL,
	[QtdeVendas] [varchar](50) NULL,
	[PrecoCusto] [varchar](50) NULL,
	[PrecoVenda] [varchar](50) NULL,
	[VendaBruta] [varchar](50) NULL,
	[Desconto] [varchar](50) NULL,
	[VendaLiquida] [varchar](50) NULL,
	[CustoVenda] [varchar](50) NULL,
	[Lucro] [varchar](50) NULL,
	[DataVenda] [date] NULL,
) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.025

**CRIANDO TABELAS DIMENSÕES**

In [65]:
--CRIANDO DIMENSAO D_SEGMENTO

IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_SEGMENTO' AND xtype='U')
CREATE TABLE [dbo].[D_SEGMENTO](
	[ID_SEGMENTO] [int] IDENTITY(1,1) NOT NULL,
	[SEGMENTO] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[ID_SEGMENTO] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO D_PRODUTO

IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_PRODUTO' AND xtype='U')
CREATE TABLE [dbo].[D_PRODUTO](
	[ID_PRODUTO] [int] IDENTITY(1,1) NOT NULL,
	[PRODUTO] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[ID_PRODUTO] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO D_PAIS

IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_PAIS' AND xtype='U')
CREATE TABLE [dbo].[D_PAIS](
	[ID_PAIS] [int] IDENTITY(1,1) NOT NULL,
	[PAIS] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[ID_PAIS] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO D_TipoDesconto

IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_TipoDesconto' AND xtype='U')
CREATE TABLE [dbo].[D_TipoDesconto](
	[ID_TipoDesconto] [int] IDENTITY(1,1) NOT NULL,
	[TipoDesconto] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[ID_TipoDesconto] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.054

**CRIANDO TABELA FATO**

In [66]:
--CRIANDO TABELA FATO_VENDA

IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='FATO_VENDA' AND xtype='U')
CREATE TABLE FATO_VENDA
(
ID_VENDA  [int] PRIMARY KEY NOT NULL,
ID_PAIS  [int] NULL,
ID_PRODUTO  [int] NULL,
ID_SEGMENTO  [int] NULL,
ID_TipoDesconto  [int] NULL,
DataVenda  DATE,
PrecoCusto  DECIMAL(10,2) NULL,
QtdeVendas  DECIMAL(10,2) NULL,
CustoVenda  DECIMAL(10,2) NULL,
Desconto DECIMAL(10,2) NULL,
Lucro DECIMAL(10,2) NULL,
PrecoVenda DECIMAL(10,2) NULL,
VendaBruta DECIMAL(10,2) NULL,
VendaLiquida DECIMAL(10,2) NULL
)

Commands completed successfully.

Total execution time: 00:00:00.018

**TRUNCANDO OS DADOS DA TABELA**

In [1]:
TRUNCATE TABLE STG_VENDAS

Commands completed successfully.

Total execution time: 00:00:00.035

**CARREGANDO DADOS NA TABELA STAGING COM BULK INSERT**

In [3]:
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  

BULK INSERT STG_VENDAS
FROM 'C:\DWE2022\Desafio_aluna28\Exercicio_06\BW-Desafio\BaseDadosDesafio.csv'
WITH        (
	codepage='65001',  -- { 'ACP' | 'OEM' | 'RAW' | 'code_page' } ] 65001
    DATAFILETYPE = 'char',   --      { 'char' | 'native'| 'widechar' | 'widenative' } ]         
	fieldterminator=';',               
	rowterminator='\n',               
	maxerrors = 0,               
	fire_triggers,             
    firstrow = 2            
	     ) 
END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA BULK STG',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA BULK STG',
		@VDATA_INI,
		GETDATE()
		)
IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(700 rows affected)

(1 row affected)

Total execution time: 00:00:00.101

**CARGA DIMENSÃO SEGMENTO**

In [4]:
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA SEGMENTO'
BEGIN TRY  


MERGE D_SEGMENTO AS Destino

USING (SELECT DISTINCT Segmento FROM STG_VENDAS) AS Origem

ON Destino.SEGMENTO = Origem.Segmento

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET SEGMENTO = Origem.Segmento

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (SEGMENTO) VALUES (Origem.Segmento);

 END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO     

(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.079

**CARGA DIMENSÃO PRODUTO**

In [5]:
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA PRODUTO'
BEGIN TRY  


MERGE D_PRODUTO AS Destino

USING (SELECT DISTINCT Produto FROM STG_VENDAS) AS Origem

ON Destino.PRODUTO = Origem.Produto

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET PRODUTO = Origem.Produto

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (PRODUTO) VALUES (Origem.Produto);

 END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO     

(6 rows affected)

(1 row affected)

Total execution time: 00:00:00.073

**CARGA DIMENSÃO PAIS**

In [6]:
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA PAIS'
BEGIN TRY  


MERGE D_PAIS AS Destino

USING (SELECT DISTINCT Pais FROM STG_VENDAS) AS Origem

ON Destino.PAIS = Origem.Pais

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET PAIS = Origem.Pais

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (PAIS) VALUES (Origem.Pais);

 END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO     

(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.067

**CARGA DIMENSÃO TIPO DE DESCONTO**

In [7]:
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA TIPO DESCONTO'
BEGIN TRY  


MERGE D_TipoDesconto AS Destino

USING (SELECT DISTINCT TipoDesconto FROM STG_VENDAS) AS Origem

ON Destino.TipoDesconto = Origem.TipoDesconto

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET TipoDesconto = Origem.TipoDesconto

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (TipoDesconto) VALUES (Origem.TipoDesconto);

 END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO     

(4 rows affected)

(1 row affected)

Total execution time: 00:00:00.035

**CARGA FATO**

In [8]:
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA FATO'
BEGIN TRY  

MERGE FATO_VENDA AS Destino

USING (
SELECT  a.ID_VENDA,
        b.ID_PAIS, 
		c.ID_PRODUTO,  
		d.ID_SEGMENTO,  
		e.ID_TipoDesconto,   
		a.DataVenda,  
		CAST(REPLACE(a.PrecoCusto,',','.') AS decimal(10,2)) AS PrecoCusto, 
		CAST(REPLACE(a.QtdeVendas,',','.') AS decimal(10,2)) AS QtdeVendas,
		CAST(REPLACE(a.CustoVenda,',','.') AS decimal(10,2)) AS CustoVenda,
		CAST(REPLACE(a.Desconto,',','.') AS decimal(10,2)) AS Desconto,
		CAST(REPLACE(a.Lucro,',','.') AS decimal(10,2)) AS Lucro,
		CAST(REPLACE(a.PrecoVenda,',','.') AS decimal(10,2)) AS PrecoVenda,
		CAST(REPLACE(a.VendaBruta,',','.') AS decimal(10,2)) AS VendaBruta,
		CAST(REPLACE(a.VendaLiquida,',','.') AS decimal(10,2)) AS VendaLiquida
 FROM STG_VENDAS a
	LEFT JOIN D_PAIS b
	ON a.Pais=B.PAIS

	LEFT JOIN D_PRODUTO c
	ON a.Produto=c.PRODUTO

	LEFT JOIN D_SEGMENTO d
	ON a.Segmento=d.SEGMENTO

	LEFT JOIN D_TipoDesconto e
	ON a.TipoDesconto=e.TipoDesconto

	) AS Origem

	ON Destino.ID_VENDA = Origem.ID_VENDA

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET  
                ID_PAIS=origem.ID_PAIS, 
                ID_PRODUTO=origem.ID_PRODUTO,
                ID_SEGMENTO=origem.ID_SEGMENTO,
                ID_TipoDesconto=origem.ID_TipoDesconto,
                DataVenda=origem.DataVenda,
                PrecoCusto=origem.PrecoCusto,
                QtdeVendas=origem.QtdeVendas,
                CustoVenda=origem.CustoVenda,
				Desconto=origem.Desconto,
				Lucro=origem.Lucro,
				PrecoVenda=origem.PrecoVenda,
				VendaBruta=origem.VendaBruta,
				VendaLiquida=origem.VendaLiquida

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
   INSERT  
           (ID_VENDA,
            ID_PAIS,
           ID_PRODUTO,
           ID_SEGMENTO,
           ID_TipoDesconto,
           DataVenda,
           PrecoCusto,
           QtdeVendas,
           CustoVenda,
           Desconto,
           Lucro,
           PrecoVenda,
           VendaBruta,
           VendaLiquida)

	VALUES
           (origem.ID_VENDA,
           origem.ID_PAIS, 
           origem.ID_PRODUTO,
           origem.ID_SEGMENTO,
           origem.ID_TipoDesconto,
		   origem.DataVenda,
		   origem.PrecoCusto,
		   origem.QtdeVendas,
		   origem.CustoVenda,
		   origem.Desconto,
		   origem.Lucro,
		   origem.PrecoVenda,
		   origem.VendaBruta,
		   origem.VendaLiquida);
           

END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

	END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
           [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(700 rows affected)

(1 row affected)

Total execution time: 00:00:00.149